# **ETAPA 3 - TRANSFORMAÇÃO CAMADA SILVER**

<br>

---

<br>

### TODO
- Testar funções trim e not null, e ajustar conforme necessário

<br><br>

---

<br>

### Parte 1 - **Verificação de catálogos**

In [0]:
# Criação do catalog
spark.sql("CREATE CATALOG IF NOT EXISTS data_ex")

# Criação do schema
spark.sql("CREATE SCHEMA IF NOT EXISTS data_ex.desafio")

# Criação dos volumes
spark.sql("CREATE VOLUME IF NOT EXISTS data_ex.desafio.lhdw")
spark.sql("CREATE VOLUME IF NOT EXISTS data_ex.desafio.bronze")
spark.sql("CREATE VOLUME IF NOT EXISTS data_ex.desafio.silver")
spark.sql("CREATE VOLUME IF NOT EXISTS data_ex.desafio.gold")


In [0]:
# Importar as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("Transformação Data Silver") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Define um número fixo de partições para shuffle, melhorando o paralelismo                 
# Define o tamanho máximo de partições para evitar muitos arquivos pequenos        
# Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita    
# Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

# Definir caminhos de armazenamento no Data Lake
# Ler dados na Bronze e Armazenar Silver

bronze_path = "/Volumes/data_ex/desafio/bronze"
silver_path = "/Volumes/data_ex/desafio/silver"



In [0]:
# Ler dados da camada Bronze
df_bronze = spark.read.format("parquet").load(bronze_path)
#display(df_bronze)

In [0]:
# TODO: Testar função de trim
for field in df_bronze.schema.fields:
    if isinstance(field.dataType, StringType):
        df_silver = df_bronze.withColumn(field.name, trim(col(field.name)))

In [0]:
from pyspark.sql.functions import format_number

# Realizar transformações necessárias
df_silver = df_silver.withColumn("Data", to_date(col("Data"), "yyyy-MM-dd")) \
                     .withColumn("Email", lower(expr("regexp_replace(split(EmailNome, ':')[0], '[()]', '')"))) \
                     .withColumn("Nome", expr("split(split(EmailNome, ':')[1], ', ')")) \
                     .withColumn("Nome", expr("concat(Nome[1], ' ', Nome[0])")) \
                     .withColumn("Cidade", expr("split(Cidade, ',')[0]")) \
                     .withColumn("TotalVendas", format_number(col("PrecoUnitario") * col("Unidades"),2)) \
                     .withColumn("PrecoUnitario", format_number(col("PrecoUnitario"), 2)) \
                     .withColumn("CustoUnitario", format_number(col("CustoUnitario"), 2)) \
                     .drop("EmailNome")\
                     .drop("IdCampanha")   
                     

display(df_silver)


In [0]:
# TODO: Testar função Not Null
df_silver = df_silver.filter(col("cst_id").isNotNull())

## ETAPA DO BRONZE - TEMPLATE VIEW

Esta função abaixo não faz parte da etapa silver

In [0]:
### ETAPA DO BRONZE - TEMPLATE VIEW

# Criando uma lista para armazenar os Sparks DataFrames
spark_df = []

# Criação do Spark DataFrame, leitura dos csv, aplicação dos schemas e escrita em parquet 
def bronze_work(csv_file, schemas):
    for file, schema in zip(csv_files, schemas):
        # Tenta ler os arquivos csv em lhdw_path
        try:
            # Se schema não for vendas, aplica o schema correspondente a file
            if schema is not "schema_vendas":
                df = spark.read.csv(f"{lhdw_path}/{file}", header=True, schema=schema)
                print(f"Leitura do arquivo {file} bem suceddida.")
            
            # Se for vendas, deve repetir a leirura do arquivo n vezes
            else:
                # Tenta ler o caminho no qual vendas esta
                try:
                    paths = dbutils.fs.ls(lhdw_path)
                    # Conta o numero csv vendas existentes
                    n = len([n for n in paths if n.name.startswith("vendas")])
                    # Aplica um loop para ler todos os arquivos vendas
                    for i in range(1, n+1):
                        try:
                            df = spark.read.csv(f"{lhdw_path}/vendas_part{i}.csv", header=True, schema=schema)
                            print(f"Leitura do arquivo vendas_part{i}.csv bem sucedida.")
                        except Exception as e:
                            print(f"Erro ao ler o arquivo {lhdw_path}/vendas_part{i}.csv: {e}")
                except Exception as e:
                    print("ERRO")
                    print(f"Erro ao ler o caminho no qual no qual vendas esta {e}")
            
        except Exception as e:
            print(f"Erro ao ler o arquivo {file}: {e}")

bronze_work(csv_files, schemas)